In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import *
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import random

In [3]:
train= pd.read_csv("E:/TookiTaki/My work/Kaggle/Shelter Animal Outcomes/train.csv")
test= pd.read_csv("E:/TookiTaki/My work/Kaggle/Shelter Animal Outcomes/test.csv")

In [4]:
test.AgeuponOutcome.unique()

array(['10 months', '2 years', '1 year', '4 months', '3 years', '13 years',
       '6 months', '3 months', '5 months', '5 years', '4 years', '3 weeks',
       '1 week', '2 months', '9 months', '4 days', '2 weeks', '7 months',
       '9 years', '1 month', '12 years', '1 weeks', '6 days', '6 years',
       '8 years', '8 months', '4 weeks', '7 years', '10 years', '5 days',
       '17 years', '3 days', '15 years', '11 years', '14 years',
       '11 months', nan, '2 days', '1 day', '16 years', '20 years',
       '0 years', '22 years', '18 years', '5 weeks', '19 years'], dtype=object)

In [5]:
train.AgeuponOutcome.unique()

array(['1 year', '2 years', '3 weeks', '1 month', '5 months', '4 years',
       '3 months', '2 weeks', '2 months', '10 months', '6 months',
       '5 years', '7 years', '3 years', '4 months', '12 years', '9 years',
       '6 years', '1 weeks', '11 years', '4 weeks', '7 months', '8 years',
       '11 months', '4 days', '9 months', '8 months', '15 years',
       '10 years', '1 week', '0 years', '14 years', '3 days', '6 days',
       '5 days', '5 weeks', '2 days', '16 years', '1 day', '13 years', nan,
       '17 years', '18 years', '19 years', '20 years'], dtype=object)

In [26]:
train.SexuponOutcome=train.SexuponOutcome.str.replace('Neutered Male','Male')
train.SexuponOutcome= train.SexuponOutcome.str.replace('Intact Male','Male')
train.SexuponOutcome=train.SexuponOutcome.str.replace('Spayed Female','Female')
train.SexuponOutcome=train.SexuponOutcome.str.replace('Intact Female','Female')

In [39]:
test.Breed.unique()

array(['Labrador Retriever Mix', 'German Shepherd/Siberian Husky',
       'Domestic Shorthair Mix', 'Collie Smooth Mix',
       'Miniature Poodle Mix', 'Beagle Mix', 'Domestic Medium Hair Mix',
       'Cairn Terrier', 'Pit Bull Mix',
       'Flat Coat Retriever/German Shepherd', 'Pointer/Labrador Retriever',
       'Doberman Pinsch Mix', 'English Bulldog', 'Norwich Terrier Mix',
       'Black Mouth Cur/Boxer', 'Pomeranian Mix',
       'Chihuahua Shorthair/Dachshund', 'Australian Cattle Dog Mix',
       'Miniature Poodle/Lhasa Apso', 'Border Terrier Mix',
       'Whippet/Rat Terrier', 'Japanese Chin Mix',
       'Labrador Retriever/Doberman Pinsch', 'Domestic Longhair Mix',
       'Black/Tan Hound Mix', 'Siamese Mix', 'Catahoula Mix',
       'Plott Hound/Labrador Retriever',
       'Labrador Retriever/American Foxhound', 'Bombay Mix',
       'Chihuahua Shorthair Mix', 'Boxer Mix',
       'Anatol Shepherd/Labrador Retriever', 'Cardigan Welsh Corgi Mix',
       'Pug/Italian Greyhound', 'D

In [4]:
train.Name.replace(np.NaN,'rajat',inplace=1)
train.SexuponOutcome.replace(np.NaN,'',inplace=1)
train.AgeuponOutcome.replace(np.NaN,'0',inplace=1)
train.drop('OutcomeSubtype',axis=1,inplace= True)
train.count()

AnimalID          26729
Name              26729
DateTime          26729
OutcomeType       26729
AnimalType        26729
SexuponOutcome    26729
AgeuponOutcome    26729
Breed             26729
Color             26729
dtype: int64

In [5]:
def get_digit(string2):
    num=''
    for char in string2:
        if char.isdigit():
            num+=char
    return num

In [6]:
def time(string):
    if 'year' in string:
        return 365* int(get_digit(string))
    elif 'month'in string:
        return 30* int(get_digit(string))
    elif 'week'in string:
        return 7* int(get_digit(string))
    elif 'day' in string:
        return int(get_digit(string))
    else:
        return 0

In [7]:
train['T_days']=train.AgeuponOutcome.apply(lambda x :time(x)).values

In [8]:
train[train.T_days.isnull()]

,AnimalID,Name,DateTime,OutcomeType,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,T_days


In [9]:
test.Name.replace(np.NaN,'rajat',inplace=1)
test.AgeuponOutcome.replace(np.NaN,'0',inplace=1)
test.count()

ID                11456
Name              11456
DateTime          11456
AnimalType        11456
SexuponOutcome    11456
AgeuponOutcome    11456
Breed             11456
Color             11456
dtype: int64

In [10]:
test['T_days']=test.AgeuponOutcome.apply(lambda x :time(x)).values
test[test.T_days.isnull()]

,ID,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,T_days


In [11]:
train['Difference']='Train' 
test['Difference']='Test'
fullData = pd.concat([train,test],axis=0)
le_Type = preprocessing.LabelEncoder()
fullData.Name= le_Type.fit_transform(fullData.Name)
fullData.AnimalType=le_Type.fit_transform(fullData.AnimalType)
fullData.SexuponOutcome= le_Type.fit_transform(fullData.SexuponOutcome)
fullData.Breed=le_Type.fit_transform(fullData.Breed)
fullData.Color=le_Type.fit_transform(fullData.Color)
train=fullData[fullData['Difference']=='Train']
test=fullData[fullData['Difference']=='Test']
train.DateTime=pd.to_datetime(train.DateTime)
test.DateTime=pd.to_datetime(test.DateTime)
train['month']=train.DateTime.dt.month
train['year']= train.DateTime.dt.year
train['days']=train.DateTime.dt.day
test['month']=test.DateTime.dt.month
test['year']= test.DateTime.dt.year
test['days']=test.DateTime.dt.day
train.drop('ID',axis=1, inplace=True)

C:\Users\Rajat\Anaconda3\lib\site-packages\pandas\core\generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\Rajat\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Rajat\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [12]:
test.drop('AnimalID',axis=1,inplace= True)
test.drop('OutcomeType',axis=1,inplace=True)
test.count()

C:\Users\Rajat\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Rajat\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


AgeuponOutcome    11456
AnimalType        11456
Breed             11456
Color             11456
DateTime          11456
Difference        11456
ID                11456
Name              11456
SexuponOutcome    11456
T_days            11456
month             11456
year              11456
days              11456
dtype: int64

In [13]:
ID_col=['AnimalID','ID']
target_col=['OutcomeType']
cat_col=['DateTime','Difference','AgeuponOutcome']
features=list(set(list(fullData.columns))-set(ID_col)-set(target_col)-set(cat_col))

In [28]:
cv=[]
lst=[]
kf=cross_validation.KFold(len(train),n_folds=5,random_state=0)
for idx1,idx2 in kf:
    x_train,x_cv=train[features].iloc[idx1],train[features].iloc[idx2]
    y_train,y_cv=train.OutcomeType.iloc[idx1],train.OutcomeType.iloc[idx2]
    clf = svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
    clf.fit(x_train,y_train)
    cv.extend(clf.predict_log_proba(x_cv))
    lst.append(clf.predict_proba(test[features]))
    print(metrics.accuracy_score(clf.predict(x_cv),y_cv))
#     random.seed(100)
#     rf = RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=25, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False, class_weight=None)
#     rf.fit(x_train, y_train)
#     cv.extend(rf.predict_proba(x_cv))
#     lst.append(rf.predict_proba(test[features]))
#     print(metrics.accuracy_score(rf.predict(x_cv),y_cv))


0.53722409278
0.524130190797
0.530864197531
0.528058361392
0.524415341441


In [24]:
cv

['Adoption',
 'Adoption',
 'Adoption',
 'Transfer',
 'Adoption',
 'Adoption',
 'Adoption',
 'Transfer',
 'Adoption',
 'Adoption',
 'Transfer',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Transfer',
 'Transfer',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Euthanasia',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Transfer',
 'Adoption',
 'Adoption',
 'Adoption',
 'Transfer',
 'Adoption',
 'Transfer',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Transfer',
 'Adoption',
 'Adoption',
 'Transfer',
 'Adoption',
 'Transfer',
 'Adoption',
 'Adoption',
 'Transfer',
 'Adoption',
 'Transfer',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Transfer',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption',
 'Adoption

In [25]:
lst

[array(['Adoption', 'Adoption', 'Adoption', ..., 'Transfer', 'Adoption',
        'Adoption'], dtype=object),
 array(['Adoption', 'Adoption', 'Adoption', ..., 'Transfer', 'Adoption',
        'Adoption'], dtype=object),
 array(['Adoption', 'Adoption', 'Adoption', ..., 'Transfer', 'Adoption',
        'Adoption'], dtype=object),
 array(['Adoption', 'Adoption', 'Adoption', ..., 'Transfer', 'Adoption',
        'Adoption'], dtype=object),
 array(['Adoption', 'Adoption', 'Adoption', ..., 'Transfer', 'Adoption',
        'Adoption'], dtype=object)]

In [22]:
Id_dummy=test.ID
Id_new=train.AnimalID
df1=pd.DataFrame({'Id': Id_dummy, 'OutcomeType': np.array(lst)})
df2=pd.DataFrame({'Id': Id_new,'OutcomeType': np.array(cv)})


Exception: Data must be 1-dimensional

In [133]:
#df1.to_csv("C:/Users/Rajat/Desktop/RandomForestShelter.csv",index= False)
